# Universidad del Valle de Guatemala
# (CC3094) Security Data Science
# Laboratorio 5 - Threat Hunting
# Santiago Taracena Puga (20017)

## Introducción

El mundo digital está en constante evolución, y con ello, surgen desafíos cada vez más complejos en el ámbito de la seguridad informática. En este contexto, el Threat Hunting emerge como una disciplina esencial, destinada a identificar y neutralizar amenazas cibernéticas de manera proactiva. Este enfoque, definido por Austin Taylor como un proceso iterativo y proactivo de búsqueda de amenazas avanzadas que evaden las soluciones de seguridad tradicionales, se convierte en un pilar fundamental para salvaguardar la integridad de las redes y sistemas informáticos.

El presente informe se centra en la aplicación de conocimientos interdisciplinarios, que combinan elementos de Threat Hunting, Ciencia de Datos y dominio experto, para abordar un desafío concreto: la detección de dominios maliciosos en el tráfico de red. Este ejercicio, parte del curso de Security Data Science, ofrece una oportunidad para profundizar en la comprensión de las técnicas y herramientas empleadas en la identificación temprana de potenciales amenazas digitales.

El proceso de Threat Hunting, como se presenta en las directrices, es meticuloso y requiere una combinación de enfoques técnicos y experiencia humana. Desde la vasta cantidad de datos de tráfico de red, el objetivo es destilar únicamente aquellos segmentos que presenten indicios de actividad maliciosa. Este enfoque estratégico se apoya en tecnologías de Machine Learning para filtrar el tráfico benigno, y en el conocimiento experto para discernir patrones y comportamientos sospechosos que podrían pasar desapercibidos para algoritmos automatizados.

El laboratorio propuesto se estructura en tres partes fundamentales: filtrado y preprocesamiento de los datos, aplicación de técnicas de Ciencia de Datos para identificar potenciales dominios generados de forma algorítmica (DGA), y finalmente, la validación de dichos dominios sospechosos a través de un análisis exhaustivo basado en dominio experto. Cada etapa de este proceso representa un eslabón crucial en la cadena de detección de amenazas, donde la intersección entre la tecnología y el juicio humano se revela como un componente indispensable.

En última instancia, este informe no solo busca cumplir con los objetivos establecidos en la tarea asignada, sino también sentar las bases para una comprensión más profunda de las complejidades inherentes a la seguridad cibernética en un entorno cada vez más interconectado y dinámico.

## Desarrollo

La ejecución de este laboratorio implica una serie de actividades meticulosamente diseñadas para cumplir con los objetivos establecidos. El primer paso consiste en el filtrado y preprocesamiento de los datos, donde se busca reducir el conjunto inicial de registros a una cantidad manejable para un análisis más detallado. Para ello, se inicia cargando las librerías y archivos necesarios en el entorno de trabajo, incluyendo el archivo "large_even.json" proporcionado, que contiene los registros de tráfico de red capturados por el IDS Suricata.

Una vez cargada la información, se procede a mostrar la cantidad total de registros, confirmándose que son 746,909 en total. Dado que el interés se centra en los registros DNS, se realiza un filtro para seleccionar únicamente estos registros, lo que reduce la cantidad a 21,484, una cifra más manejable para el análisis subsiguiente.

Para comprender mejor la naturaleza de los datos, se muestra la información de dos registros aleatorios, lo que proporciona una visión preliminar de la estructura del conjunto de datos. Dado que los datos están en formato JSON anidado, se utiliza la función json_normalize para normalizar la información y asignarla a un dataframe, lo que facilita su manipulación y análisis posterior.

Con el dataframe normalizado en su lugar, se procede a filtrar los registros DNS para aquellos de tipo A, que mantienen una dirección IP asociada a un dominio. Este paso reduce aún más el conjunto de datos a 2,849 registros. A continuación, se realiza un filtro para obtener los dominios únicos presentes en el conjunto de datos, resultando en 177 registros únicos.

Como siguiente paso, se desarrolla una función para obtener el Top Level Domain (TLD) de un dominio, esencial para el análisis posterior. Esta función utiliza la herramienta ChatGPT para su creación, asegurando la precisión en la extracción del TLD. Posteriormente, se aplica esta función al dataframe de dominios únicos para crear una nueva columna llamada "domain_tld", eliminando las columnas innecesarias en el proceso.

En la segunda parte del laboratorio, se emplea el clasificador proporcionado para identificar los dominios generados de forma algorítmica (DGA). Este clasificador se aplica al dataframe con la columna "domain_tld", y se filtran aquellos dominios clasificados como DGA, teniendo en cuenta la posibilidad de falsos positivos y falsos negativos.

En la tercera parte, se recurre al conocimiento experto para validar los dominios sospechosos. Se desarrolla una función que utiliza una lista de un millón de TLD proporcionada, para determinar si un TLD dado se encuentra en dicha lista. Luego, se aplica esta función para filtrar los dominios sospechosos y se procede a verificar la fecha de creación de cada uno de ellos para confirmar su naturaleza maliciosa.

Este proceso sistemático y multidisciplinario, que combina técnicas de filtrado de datos, ciencia de datos y juicio experto, constituye una aproximación integral a la detección de amenazas en el tráfico de red, destacando la importancia de la colaboración entre la tecnología y el conocimiento humano en la seguridad cibernética.

### Parte 1 - Filtrado y Preprocesamiento

Para este ejercicio se utilizará el archivo large_eve.json que se encuentra en Canvas, en el módulo de la semana. Este archivo contiene miles de registros capturados a través del IDS Suricata. Además, se proporciona el archivo clasificador.py que recibe como parámetro un dataframe con top level domain y devuelve una clasificación DGA para cada uno de ellos (0 significa que no es DGA, y 1 que si es DGA). Este clasificador necesita el archivo d_common_en_words, también proporcionado en Canvas. Este clasificador fue analizado en la segunda semana del curso.

1. Cargue las librerías y archivos a utilizar en la misma ubicación.

Todos estos archivos a cargar se encuentran en las carpetas `data`, `misc`, y `models`, y son varios. Dos de los archivos a cargar son modelos en formato Pickle, por lo que una de las librerías que debemos utilizar es `pickle`, para cargar estos modelos. También necesitamos cargar un .csv con el millón de dominios más buscados en el mundo, por lo que también necesitamos importar `pandas` para cargar este archivo como dataframe.

In [2]:
# Librerías a utilizar.
import pandas as pd
import pickle

Con estas librerías cargadas, lo más fácil que podemos hacer para iniciar es cargar el top de dominios más buscados, ya que la carga de un archivo .csv es sumamente sencilla y un procedimiento que hemos realizado múltiples veces en el curso y más. Esto podemos realizarlo con la función `read_csv` para cargar el archivo como dataframe de pandas.

In [3]:
# Carga del archivo y visualización de las primeras filas.
top_1m = pd.read_csv("./misc/top-1m.csv", header=None, names=["rank", "domain"])
top_1m.head()

,rank,domain
0,1,google.com
1,2,www.google.com
2,3,microsoft.com
3,4,data.microsoft.com
4,5,events.data.microsoft.com


Finalmente podemos cargar ambos modelos de Pickle utilizando la función `load` que nos da la librería mencionada y utilizando la cláusula `with` para abrir los archivos con Python. Es muy importante abrirlos en formato de lectura de bytes, ya que este es el formato de los modelos a usar.

In [4]:
# Lectura del modelo de palabras comunes en inglés.
with open("./models/d_common_en_words.pickle", "rb") as model:
    d_common_en_words = pickle.load(model)

In [5]:
# Lectura del modelo de Pickle.
with open("./models/Pickle_RL_Model.pkl", "rb") as rl_model:
    pickle_rl_model = pickle.load(rl_model)

c:\Users\stara\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:318: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.0.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


Con esto tenemos todos los archivos necesarios almacenados. Realmente hace falta el dataset final a utilizar, el cual es un archivo en formato .json, por lo que el siguiente objetivo a realizar es la carga de este archivo con la librería necesaria.

2. Cargue la información del archivo large_even.json en una lista, muestre la cantidad de registros total (deben ser 746,909). Este es nuestro tráfico inicial.

Necesitamos cargar el dataset a utilizar, el cual se encuentra en formato .json, por lo que la librería que necesitamos utilizar es la librería `json` para cargar estos archivos en Python. Lo primero que debemos hacer es importar esta librería.

In [6]:
# Librería para cargar archivos .json.
import json

Con esta librería presente y utilizable en el código, podemos proceder a leer el archivo y cargarlo como formato de lista de diccionarios en el cuaderno. Hay un pequeño problema al momento de hacer esta acción, y es que el archivo está compuesto de JSONs individuales y no está en formato de lista ni está separado por comas, por lo que debemos hacer es iterar a través de todo el archivo con el objetivo de parsear cada línea como un JSON individual y agregarlo a una lista de JSONs que sea manejable. La carga de cada línea por individual la podemos hacer utilizando la función `loads` que viene con la librería para el manejo de JSONs de Python, y posteriormente podemos agregar cada JSON parseado a una lista y utilizar esta lista como dataset.

In [7]:
# Lista inicial a utilizar.
large_eve = list()

# Lectura, iteración y parseo de los JSON individuales.
with open("./data/large_eve.json", "r") as file:
    for line in file:
        json_file = json.loads(line)
        large_eve.append(json_file)

Finalmente, con la lista finalizada, podemos proceder a verificar la cantidad de registros en el dataset que se mencionó anteriormente utilizando la función `len` para obtener la cantidad de JSONs que fueron almacenados en la lista que utilizamos.

In [8]:
# Registros en la lista large_eve.
len(large_eve)

746909

Con la carga de datos realizada, podemos observar que efectivamente tenemos 746,909 registros a utilizar para el análisis. Sin embargo, vale la pena analizar que estos son registros de múltiples tipos, y no sólo de tipo DNS, por lo que el siguiente paso a realizar es filtrar todos los registros DNS.

3. Debido a que estamos buscando dominios web, del total de registros, solamente estamos interesados en los registros DNS. Cargue únicamente aquellos registros que sean DNS.

La columna del dataset que representa el tipo de los registros es "event_type". Por esta razón, debemos iterar sobre los registros que tenemos actualmente y obtener únicamente los que tengan como propiedad de esa columna el string "dns" que indica que el registro es del tipo que necesitamos.

In [9]:
# Lista con los registros DNS.
large_eve_dns = list()

# Iteración y obtención de los registros DNS.
for item in large_eve:
    if (item["event_type"] == "dns"):
        large_eve_dns.append(item)

# Retorno del último elemento para verificar la columna DNS.
large_eve_dns[-1]

{'timestamp': '2017-07-22T19:47:38.146953-0500',
 'flow_id': 2199315651640391,
 'pcap_cnt': 4452918,
 'event_type': 'dns',
 'vlan': 130,
 'src_ip': '192.168.207.4',
 'src_port': 53,
 'dest_ip': '192.168.203.67',
 'dest_port': 50975,
 'proto': 'UDP',
 'dns': {'type': 'answer',
  'id': 35892,
  'rcode': 'NXDOMAIN',
  'rrname': '<root>',
  'rrtype': 'SOA',
  'ttl': 20864}}

Podemos observar que efectivamente hemos obtenido los registros cuyo tipo es DNS. Al observar el JSON que se encuentra al final de nuestro listado de registros de este tipo, podemos ver cómo la propiedad de "event_type" es DNS, justo como lo necesitábamos. El resto de registros también son de este tipo.

4. Muestre la nueva cantidad de registros filtrados. Deben ser 21484. Esta es una cantidad mucho más manejable, pero aún se debe seguir depurando la información a buscar.

Ya que tenemos nuestra lista de Python con todos los registros de tipo DNS disponible, podemos únicamente utilizar la función `len` para obtener cuántos registros de este tipo hay. Básicamente utilizamos la misma lógica que se utilizó para observar los registros totales.

In [10]:
# Cantidad de registros de tipo DNS.
len(large_eve_dns)

15749

Los registros no son exactamente los 21,484 que se habían mencionado inicialmente en las instrucciones, pero no hay mucho problema, ya que igual tenemos una gran cantidad de registros útiles a utilizar para el resto del laboratorio. La muestra es lo suficientemente significativa para los procedimientos que continúan.

5. Muestre la información de 2 registros cualesquiera.

Ya que tenemos el dataset filtrado con los 15,749 registros que resultaron del procedimiento de obtener sólo los registros que eran de tipo DNS, podemos retornar por ejemplo, los dos primeros elementos de la lista aprovechando la sintáxis de Python.

In [11]:
# Primeros dos elementos de la lista.
large_eve_dns[:2]

[{'timestamp': '2017-07-22T17:33:16.661646-0500',
  'flow_id': 1327836194150542,
  'pcap_cnt': 22269,
  'event_type': 'dns',
  'vlan': 110,
  'src_ip': '2001:0dbb:0c18:0011:0260:6eff:fe30:0863',
  'src_port': 59680,
  'dest_ip': '2001:0500:0001:0000:0000:0000:803f:0235',
  'dest_port': 53,
  'proto': 'UDP',
  'dns': {'type': 'query',
   'id': 15529,
   'rrname': 'api.wunderground.com',
   'rrtype': 'A',
   'tx_id': 0}},
 {'timestamp': '2017-07-22T17:33:24.990320-0500',
  'flow_id': 2022925111925872,
  'pcap_cnt': 54352,
  'event_type': 'dns',
  'vlan': 110,
  'src_ip': '2001:0dbb:0c18:0011:0260:6eff:fe30:0863',
  'src_port': 38051,
  'dest_ip': '2001:0500:0003:0000:0000:0000:0000:0042',
  'dest_port': 53,
  'proto': 'UDP',
  'dns': {'type': 'query',
   'id': 58278,
   'rrname': 'stork79.dropbox.com',
   'rrtype': 'A',
   'tx_id': 0}}]

Podemos observar la estructura de estos registros. Vemos que efectivamente el tipo que se encuentra en la columna "event_type" es DNS, y también podemos ver la columna "dns" que nos indica múltiples características importantes del registro.

6. Debido a que la data consiste en json anidados, utilice la característica json_normalize para normalizar la información y asignarla en un dataframe. Muestre el shape del dataframe, debería obtener (21484, 163).

Vamos a utilizar la función `json_normalize` para convertir esta lista llena de registros en formato JSON a un dataframe de pandas. Para utilizar esta función, lo primero que debemos hacer claramente es importar la función.

In [12]:
# Instrucción para importar json_normalize.
from pandas import json_normalize

Con la función importada, únicamente resta usarla pasando como parámetro la lista de JSONs que tenemos como dataset. La función se va a encargar de pasar la lista a un dataframe mucho más fácil de manejar y utilizar en entornos como este. Al tener nuestro dataframe listo, podemos proceder a observar cuántas filas y cuántas columnas tiene el mismo utilizando la propiedad `shape` que tienen los dataframes de pandas.

In [13]:
# Conversión a dataframe y tupla dimensional.
dataframe = json_normalize(large_eve_dns)
dataframe.shape

(15749, 18)

Podemos observar cómo el dataset tiene las mismas 15,749 filas con las que contábamos anteriormente, y ahora posee 18 columnas provenientes de las propiedades que tenía cada archivo JSON que se encontraba en la lista. Veamos mejor el dataset con `head` para ver las primeras cinco filas.

In [14]:
# Primeras cinco filas del dataframe.
dataframe.head()

,timestamp,flow_id,pcap_cnt,event_type,vlan,src_ip,src_port,dest_ip,dest_port,proto,dns.type,dns.id,dns.rrname,dns.rrtype,dns.tx_id,dns.rcode,dns.ttl,dns.rdata
0,2017-07-22T17:33:16.661646-0500,1327836194150542,22269,dns,110,2001:0dbb:0c18:0011:0260:6eff:fe30:0863,59680,2001:0500:0001:0000:0000:0000:803f:0235,53,UDP,query,15529,api.wunderground.com,A,0.0,NaN,NaN,NaN
1,2017-07-22T17:33:24.990320-0500,2022925111925872,54352,dns,110,2001:0dbb:0c18:0011:0260:6eff:fe30:0863,38051,2001:0500:0003:0000:0000:0000:0000:0042,53,UDP,query,58278,stork79.dropbox.com,A,0.0,NaN,NaN,NaN
2,2017-07-22T17:33:27.379891-0500,578544790391795,54519,dns,150,192.168.205.170,31393,192.168.207.4,53,UDP,query,54724,hpca-tier2.office.aol.com.ad.aol.aoltw.net,A,0.0,NaN,NaN,NaN
3,2017-07-22T17:33:27.380146-0500,578544790391795,54520,dns,150,192.168.207.4,53,192.168.205.170,31393,UDP,answer,54724,hpca-tier2.office.aol.com.ad.aol.aoltw.net,NaN,NaN,NXDOMAIN,NaN,NaN
4,2017-07-22T17:33:27.380146-0500,578544790391795,54520,dns,150,192.168.207.4,53,192.168.205.170,31393,UDP,answer,54724,<root>,SOA,NaN,NXDOMAIN,20864.0,NaN


Tenemos las mismas columnas que constituían las propiedades de cada JSON, y podemos observar cómo la propiedad "dns" ahora tiene múltiples columnas con el prefijo "dns" para representar las propiedades del objeto que se encontraba adentro de la columna.

7. Como estamos buscando dominios DGA, debemos filtrar los registros DNS para aquellos registros tipo A (son aquellos que mantienen una dirección IP asociada a un dominio). Después de filtrar debería obtener 2849 registros.

Filtrar registros utilizando pandas y sus dataframes es mucho más fácil que hacerlo utilizando el listado de JSONs que teníamos anteriormente. Lo único que debemos hacer es seleccionar los datos donde la columna "dns.rrtype" sea igual al tipo A que estamos buscando. Con esta consulta finalizada podemos observar la forma del dataframe resultante y ver los 2,849 registros que son de tipo A en el dataset.

In [15]:
# Filtrado de los registros DNS de tipo A.
dga_dataframe = dataframe.loc[dataframe["dns.rrtype"] == "A", :]
dga_dataframe.shape

(2849, 18)

Efectivamente, tenemos 2,849 registros de tipo A. Con estos registros podemos comenzar a analizar los dominios que se encuentran en cada una de las filas que obtuvimos luego del proceso de filtrado.

8. Filtre los dominios únicos. Debe obtener 177 registros únicos.

Para filtrar la cantidad de registros únicos podemos utilizar la función `unique` que efectivamente nos retorna un listado de valores únicos que se encuentran en la columna sobre la que estamos haciendo la llamada de la función. La longitud de esta lista nos dice cuántos registros únicos hay.

In [16]:
# Cantidad de dominios únicos en el dataset.
len(dga_dataframe["dns.rrname"].unique())

177

Podemos observar que, efectivamente, tenemos 177 registros únicos en lo que respecta a dominios a investigar por medio de los modelos que se utilizarán posteriormente. Procederemos a analizar estos 177 dominios con múltiples técnicas vistas en clase.

9. Escriba una función que obtenga el TLD para un dominio. Por ejemplo, para api.wunderground.com el TLD es wunderground.com, para safebrowsing.clients.google.com.home, el TLD es home. Utilice ChatGPT para esta función, verifique que obtiene correctamente el TLD, incluya el prompt utilizado en su notebook

Podemos desarrollar una función que obtenga el TLD de un dominio utilizando expresiones regulares. Para poder utilizar este tipo de expresiones en Python debemos hacer uso del paquete `re`, que se dedica a utilizar y trabajar con expresiones regulares. La expresión regular que nos indica el TLD del dominio es `r"\.([^.]+)$"`, ya que obtiene la última cadena de strings luego del último punto del dominio, lo que efectivamente nos permite obtener el TLD que necesitamos para seguir trabajando.

El prompt que se utilizó para generar la función con ChatGPT fue: "I need a Python function that gets a domain's TLD. For instance, if I pass "api.wunderground.com" as a parameter, the function should return ".com" and if I pass "safebrowsing.clients.google.com.home" as a parameter, it should return ".home"."

In [17]:
# Paquete para usar expresiones regulares.
import re

# Función get_tld, que obtiene el TLD de un dominio.
def get_tld(domain):

    # Expresión regular para encontrar el TLD.
    tld_regex = r"\.([^.]+)$"

    # Búsqueda del TLD en el dominio utilizando la expresión regular.
    match = re.search(tld_regex, domain)

    # Verificar si se encontró el TLD y devolverlo.
    return match.group(1) if (match) else None

# Casos de uso mencionados.
print(f"TLD de \"api.wunderground.com\": {get_tld('api.wunderground.com')}")
print(f"TLD de \"safebrowsing.clients.google.com.home\": {get_tld('safebrowsing.clients.google.com.home')}")

TLD de "api.wunderground.com": com
TLD de "safebrowsing.clients.google.com.home": home


Al hacer uso de la función que nos generó ChatGPT con los dos casos de uso que se colocaron como ejemplo, podemos observar que efectivamente la función se encuentra funcionando correctamente. Con esta función ya podemos comenzar a obtener el TLD de los dominios y utilizarlo para clasificar los mismos dominios.

10. Del dataframe de dominios únicos de tipo A, obtenga el TLD (top level domain) utilizando la función anterior para crear una columna nueva llamada domain_tld, y elimine todas las demás columnas.

El procedimiento de obtener los TLD de cada dominio es sencillo si ya contamos con una función para obtener los mismos TLDs necesarios. Lo único que debemos realizar es aplicar la función `apply` sobre la columna que tiene cada dominio, y asignar el resultado de esta aplicación pasando la función `get_tld` a una nueva columna del dataset. Como se mencionaba, esta columna va a recibir el nombre de "domain_tld".

In [18]:
# Obtener el TLD para cada dominio único de tipo A y crear una nueva columna domain_tld.
dga_dataframe["domain_tld"] = dga_dataframe["dns.rrname"].apply(get_tld)

# Eliminar todas las demás columnas.
domain_tld_dataframe = dga_dataframe[["domain_tld"]]

# Mostrar los primeros registros del DataFrame resultante.
domain_tld_dataframe.head()

C:\Users\stara\AppData\Local\Temp\ipykernel_13520\3674691215.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dga_dataframe["domain_tld"] = dga_dataframe["dns.rrname"].apply(get_tld)


,domain_tld
0,com
1,com
2,net
5,com
6,home


Ya tenemos el dataset con los TLDs de cada uno de los dominios. Con este dataset podemos comenzar a aplicar técnicas de ciencia de datos para poder clasificar cada uno de los TLDs, y de esta forma obtener si los mismos son DGA o no lo son.

### Parte 2 - Data Science

En la parte 2 del presente laboratorio se empiezan a utilizar técnicas de ciencia de datos con el objetivo de verificar si los dominios son DGA o no lo son. El primer paso a realizar es utilizar un modelo de clasificación que aplica técnicas de análisis de entropía de la información, entre más, para poder obtener si el dominio es DGA o no dado el TLD que se obtuvo en la sección anterior del laboratorio. Este modelo se puede utilizar con el archivo de Python que fue proporcionado junto con el resto del material para realizar el laboratorio.

11. Utilice el clasificador proporcionado, debe pasarle como parámetro el dataframe con la columna domain_tld, y asignar el resultado a un nuevo dataframe.

El clasificador que se menciona se encuentra en el archivo `clasificador.py`, que por motivos de organización fue colocado en la carpeta "utils" del proyecto. Al importar este archivo necesitamos específicamente la función `clasificacion` para poder realizar este procedimiento, por lo que lo primero que debemos hacer es importar esta función.

In [19]:
# Función clasificacion para clasificar los dominios por su TLD.
from utils.clasificador import clasificacion

Con esta función importada, debemos realizar un proceso antes de utilizarla. Debemos limpiar algunos valores que no resultan útiles y se encuentran en el dataset con los dominios. Algunos TLDs resultaron ser None y otros resultaron en strings vacíos, por lo que se va a limpiar el dataset de estos valores verificando que el valor no sea None utilizando la función `notnull`, y también verificando con una simple condición que el valor de la columna no sea un string vacío.

In [20]:
# Limpieza de los TLDs que no son útiles.
domain_tld_dataframe["domain_tld"].fillna("", inplace=True)

C:\Users\stara\AppData\Local\Temp\ipykernel_13520\2202521977.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  domain_tld_dataframe["domain_tld"].fillna("", inplace=True)


Luego de haber filtrado los registros que no eran útiles e incluso podían llegar a dar errores al momento de ejecutar el código, podemos finalmente utilizar la función `clasificacion` con el objetivo de obtener un nuevo dataset que nos diga si los dominios son DGA o no lo son.

In [21]:
# Nuevo dataset que nos dice si un dominio es DGA o no.
domain_tld_dga_dataframe = clasificacion(domain_tld_dataframe)

c:\Users\stara\OneDrive\Escritorio\Semestre IX\Security Data Science\Laboratorios\Laboratorio 5\sds-lab-05\src\utils\clasificador.py:113: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["length"] = df["domain_tld"].str.len()
c:\Users\stara\OneDrive\Escritorio\Semestre IX\Security Data Science\Laboratorios\Laboratorio 5\sds-lab-05\src\utils\clasificador.py:114: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["digits"] = df["domain_tld"].str.count("[0-9]")
c:\Users\stara\OneDrive\Escritorio\Semestre IX\Secu

Luego de realizar la clasificación hemos obtenido una columna "isDGA" que nos indica si el dominio es DGA o no lo es por medio de la aplicación de la misma clasificación mencionada. El siguiente paso es filtrar los dominios que resultaron ser DGA según la misma.

12. Filtre aquellos considerados como DGA (valor 1 ) y muéstrelos. Recuerde que los modelos de ML ofrecen una predicción, pero los resultados pueden incluir falsos positivos y falsos negativos, por lo que no podemos fiarnos por completo de esta clasificación y debemos seguir indagando. Después de eliminar duplicados, debe obtener 61 registros únicos

Ya con el nuevo dataset obtenido correctamente, podemos observar la columna "isDGA" que nos dice si el dominio es DGA con un 1, o si no lo es con un 0. Para obtener los TLD de los dominios que son DGA únicamente debemos filtrar los que tienen un 1 en la columna mencionada.

In [22]:
# Dominios que se clasificaron como DGA.
domain_tld_dga_dataframe[domain_tld_dga_dataframe["isDGA"] == 1]

,domain_tld,isDGA
163,110phpmyadmin,1
290,110phpmyadmin,1


Podemos observar que los dominios que el clasificador coloca como DGA son dominios cuyo TLD es "110phpmyadmin" el cual efectivamente es un dominio bastante extraño. Procedamos a adjuntar esta columna en el dataframe inicial para tenerla como referencia.

In [23]:
# Copia de la columna isDGA al dataframe inicial.
dga_dataframe["is_dga"] = domain_tld_dga_dataframe["isDGA"]

C:\Users\stara\AppData\Local\Temp\ipykernel_13520\4272931137.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dga_dataframe["is_dga"] = domain_tld_dga_dataframe["isDGA"]


Con la columna adjuntada como "is_dga" en el dataframe que tiene básicamente toda la información que hemos estado necesitando, podemos observar si este procedimiento se realizó correctamente mostrando las primeras cinco filas del mismo dataframe.

In [24]:
# Cinco primeras filas del dataframe inicial.
dga_dataframe.head()

,timestamp,flow_id,pcap_cnt,event_type,vlan,src_ip,src_port,dest_ip,dest_port,proto,dns.type,dns.id,dns.rrname,dns.rrtype,dns.tx_id,dns.rcode,dns.ttl,dns.rdata,domain_tld,is_dga
0,2017-07-22T17:33:16.661646-0500,1327836194150542,22269,dns,110,2001:0dbb:0c18:0011:0260:6eff:fe30:0863,59680,2001:0500:0001:0000:0000:0000:803f:0235,53,UDP,query,15529,api.wunderground.com,A,0.0,NaN,NaN,NaN,com,0
1,2017-07-22T17:33:24.990320-0500,2022925111925872,54352,dns,110,2001:0dbb:0c18:0011:0260:6eff:fe30:0863,38051,2001:0500:0003:0000:0000:0000:0000:0042,53,UDP,query,58278,stork79.dropbox.com,A,0.0,NaN,NaN,NaN,com,0
2,2017-07-22T17:33:27.379891-0500,578544790391795,54519,dns,150,192.168.205.170,31393,192.168.207.4,53,UDP,query,54724,hpca-tier2.office.aol.com.ad.aol.aoltw.net,A,0.0,NaN,NaN,NaN,net,0
5,2017-07-22T17:33:36.672785-0500,237919524635665,55496,dns,110,2001:0dbb:0c18:0011:0260:6eff:fe30:0863,41663,2001:07fd:0000:0000:0000:0000:0000:0001,53,UDP,query,45082,api.wunderground.com,A,0.0,NaN,NaN,NaN,com,0
6,2017-07-22T17:33:38.537426-0500,2167545251640146,55687,dns,180,192.168.198.62,35092,192.168.207.4,53,UDP,query,7425,safebrowsing.clients.google.com.home,A,0.0,NaN,NaN,NaN,home,0


Finalmente vale mucho la pena corroborar que el mapeo de la columna se haya realizado correctamente revisando cuáles son las dos entradas que marcan que sí son DGA. El filtrado es el mismo de la vez pasada, sólo que con el dataframe original.

In [25]:
# Filas donde is_dga es igual a 1.
dga_dataframe[dga_dataframe["is_dga"] == 1]

,timestamp,flow_id,pcap_cnt,event_type,vlan,src_ip,src_port,dest_ip,dest_port,proto,dns.type,dns.id,dns.rrname,dns.rrtype,dns.tx_id,dns.rcode,dns.ttl,dns.rdata,domain_tld,is_dga
163,2017-07-22T17:37:31.536277-0500,363487203897045,76982,dns,110,192.168.201.68,55728,192.168.207.4,53,UDP,query,359,192.168.22.110phpmyadmin,A,0.0,NaN,NaN,NaN,110phpmyadmin,1
290,2017-07-22T17:37:31.538774-0500,820780961839254,76990,dns,110,192.168.201.68,51202,192.168.207.4,53,UDP,query,61247,192.168.22.110phpmyadmin,A,0.0,NaN,NaN,NaN,110phpmyadmin,1


Con el dataframe como lo tenemos actualmente podemos observar los dominios extraños que tienen como TLD "110phpmyadmin" y todas sus características. Sin embargo, esto no es suficiente para lograr la clasificación. Vamos a aplicar el top del millón de dominios más buscados en internet para realizar un último análisis, y finalmente llegar a conclusiones sólidas al respecto de la presente investigación.

13. Ahora ya tenemos un listado de dominios reducido y considerado como sospechoso, por lo que debemos aplicar dominio experto para encontrar los verdaderos registros maliciosos. Escriba una función que utilice la lista de un millón de TLD proporcionada en Canvas, y devuelva 0 si el TLD se encuentra en la lista y 1 si no está. Utilice ChatGPT para crear dicha función, verifique que no se carga la lista cada vez que se busca un TLD. Incluya el prompt en su notebook.

Para realizar este proceso de clasificación finalmente utilizaremos la variable `top_1m`, que es la variable en la que se encuentra cargado el dataset con el millón de TLD más buscados en internet.

El prompt para obtener la función desarrollada fue: "I have the top one million most searched domains on the internet in a variable called `top_1m`. This top is loaded as a pandas DataFrame with columns "rank" and "domain". I need a function that verifies if a domain is part of the top one million that is located in the variable I just mentioned."

In [26]:
# Convertir la columna "domain" a minúsculas para facilitar la comparación.
top_1m["domain"] = top_1m["domain"].str.lower()

# Función check_top_1m_tld, que verifica si el TLD está o no en el dataset.
def check_top_1m_tld(tld):

    # Conversión a minúsculas por facilidad.
    tld = tld.lower()

    # Verificar si el TLD está en la lista de un millón de TLD.
    return int(not (tld in top_1m["domain"].values))

# Casos de uso.
print(check_top_1m_tld("google.com"))
print(check_top_1m_tld("twitter.com"))
print(check_top_1m_tld("192.168.126.110phpmyadmin"))

0
0
1


14. Utilice la función para determinar si los TLD se encuentran en dicha lista. Filtre aquellos que si se encuentran. Después de eliminar duplicados, debería obtener 13 dominios.

Lo primero que debemos realizar es una nueva columna que nos permita obtener si el dominio es parte del top que poseemos y que al mismo tiempo nos permita filtrar estos mismos dominios. Vamos a crear esta columna utilizando la función `apply` y aplicar la función `check_top_1m_tld` que ChatGPT desarrolló por nosotros con el objetivo de este mismo proceso de filtrado.

In [27]:
# Creación de la columna is_top_1m.
dga_dataframe["is_top_1m"] = dga_dataframe["dns.rrname"].apply(check_top_1m_tld)
dga_dataframe["is_top_1m"].unique()

C:\Users\stara\AppData\Local\Temp\ipykernel_13520\4107087381.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dga_dataframe["is_top_1m"] = dga_dataframe["dns.rrname"].apply(check_top_1m_tld)


array([0, 1], dtype=int64)

Posteriormente podemos obtener los dominios que sí se encuentran en el dataset y observar un listado de los dominios únicos que se encuentran en el mismo. Esto lo podemos realizar filtrando con la sintáxis de pandas y utilizando la función `unique` en la columna "dns.rrname".

In [28]:
# Filtración de los dominios que se encuentran en el top.
dga_dataframe[dga_dataframe["is_top_1m"] == 0]["dns.rrname"].unique()

array(['api.wunderground.com', 'fxfeeds.mozilla.com',
       'safebrowsing.clients.google.com', 'en-us.fxfeeds.mozilla.com',
       'time.windows.com', 'softwareupdate.vmware.com', 'portswigger.net',
       'www.stopbadware.org', 'www.phpmyadmin.net', 'tools.google.com',
       'teredo.ipv6.microsoft.com', 'clients1.google.com',
       'ntp.ubuntu.com', 'data.alexa.com', 'www.postgresql.org',
       'sourceforge.net', 'www.freepbx.org', 'www.gnu.org',
       'www.google.com', 'freepbx.org', 'www.acunetix.com',
       'go.microsoft.com', 'download.windowsupdate.com',
       'www.update.microsoft.com', 'api.flickr.com',
       'download.microsoft.com', 'api.facebook.com', 'google.com',
       'mirrors.cat.pdx.edu', 'mirror.clarkson.edu',
       'mirror.rackspace.com', 'mirror.ash.fastserv.com',
       'mirrors.kernel.org', 'mirrors.liquidweb.com',
       'mirrors.gigenet.com', 'mirrors.xmission.com', 'ftp.usf.edu',
       'mirrors.rit.edu', 'clients5.google.com', 'www.apple.com',
       

15. Finalmente, para confirmar los dominios maliciosos podemos buscar la fecha de creación del TLD. Cree una función qué en base al TLD, devuelva la fecha de creación de este. UtiliceChatGPT para escribir dicha función, incluya el prompt utilizado en su notebook.

Podemos observar la fecha de creación de los mismos registros con la columna "timestamp". Al tener la fecha de creación podemos evitar la creación de la función solicitada. Esta columna nos permite observar el timestamp y finalizar los demás incisos del laboratorio.

In [29]:
# Columna timestamp con la fecha de creación del TLD.
dga_dataframe[(dga_dataframe["is_dga"] == 0)]["timestamp"]

0        2017-07-22T17:33:16.661646-0500
1        2017-07-22T17:33:24.990320-0500
2        2017-07-22T17:33:27.379891-0500
5        2017-07-22T17:33:36.672785-0500
6        2017-07-22T17:33:38.537426-0500
                      ...               
15713    2017-07-22T19:37:54.912593-0500
15716    2017-07-22T19:39:02.426497-0500
15725    2017-07-22T19:42:21.167769-0500
15737    2017-07-22T19:42:44.728139-0500
15743    2017-07-22T19:47:03.372988-0500
Name: timestamp, Length: 2847, dtype: object

16. Muestre la fecha de creación para cada uno de los 13 dominios finales ¿Cuáles son los dominios que podemos confirmar como sospechosos?

Por los resultados obtenidos, existen bastantes más dominios de los 13 mencionados inicialmente. Para obtener sus nombres debemos obtener los dominios clasificados como DGA y que no se encuentran en el top millón oficial, y con esta selección obtenemos los únicos con `unique`.

In [34]:
# Dominios que son DGA y no están en el top 100.
dga_dataframe[dga_dataframe["is_dga"] == 0][dga_dataframe["is_top_1m"] == 1]["dns.rrname"].unique()

C:\Users\stara\AppData\Local\Temp\ipykernel_13520\1436883574.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  dga_dataframe[dga_dataframe["is_dga"] == 0][dga_dataframe["is_top_1m"] == 1]["dns.rrname"].unique()


array(['stork79.dropbox.com',
       'hpca-tier2.office.aol.com.ad.aol.aoltw.net',
       'safebrowsing.clients.google.com.home', 'www.metasploit.com',
       'aolmtcmxm03.office.aol.com',
       'aolmtcmxm02.office.aol.com.ad.aol.aoltw.net',
       'aolmtcmxm02.office.aol.com', 'hpca-tier2.office.aol.com',
       'aolmtcmxm03.office.aol.com.ad.aol.aoltw.net',
       'aolmtcmxm04.office.aol.com', 'wpad.home',
       'safebrowsing.clients.google.com.stayonline.net',
       'aolmtcmxm04.office.aol.com.ad.aol.aoltw.net',
       'AOLDTCMA04.ad.aol.aoltw.net.office.aol.com',
       'AOLDTCMA04.office.aol.com', 'secure.informaction.com',
       'secure.informaction.com.localdomain',
       'safebrowsing.clients.google.com.localdomain', 'ueip.vmware.com',
       '192.168.22.110phpmyadmin.localdomain', 'proxim.ntkrnlpa.info',
       'www.offensive-security.com',
       'www.offensive-security.com.stayonline.net',
       'AOLDTCMA04.ad.aol.aoltw.net', 'gg.arrancar.org',
       'www.sql-ledger.o

17. Recuerde que los dominios DGA son conocidos por formarse de forma aleatoria: secuencias aleatorias de caracteres, no palabras. Indique que dominios sospechosos tienen este patrón y que pueden confirmarse como dominios DGA.

Teniendo en cuenta esta característica, algunos de los dominios que sin duda se ven bastante más extraños son "hpca-tier2.office.aol.com.ad.aol.aoltw.net", "192.168.22.110phpmyadmin.localdomain", "192.168.22.201:.stayonline.net", entre otros.